## **0 - Introduction and Install Dependencies** (_if needed_)

If you are seeing this notebook for the first time and would like to run it, please follow these steps. Note that you will need several (>9) GB of free space in order to succesfully do the following:

1) In the same directory where this notebook is currently located on your device, make a new folder called "data\".
2) Visit this [page on Kaggle](https://www.kaggle.com/datasets/grassknoted/asl-alphabet) and download the dataset.
3) Extract the data from the compressed file into the "data\" folder, such that your current directory mirrors the following (thank you for helping me generate this figure ChatGPT):

```
root_directory
   |- exploratory_analysis.ipynb
   |- data
      |- asl_alphabet_test
         |- data...
      |- asl_alphabet_train
         |- data...
```

In [ ]:
#%pip install -U argparse
#%pip install -U tqdm
#%pip install -U scikit-learn
#%pip install -U opencv-python
#%pip install -U matplotlib
#%pip install -U tensorflow
#%pip install -U pandas
#%pip install -U numpy

## **1 - Load Libraries**

In [ ]:
# Load dependencies
from keras.layers import Conv2D, Dense, Dropout, Flatten # type: ignore
from keras.models import Sequential # type: ignore
import tensorflow as tf # type: ignore
import os
import argparse
from keras.models import Sequential, Model # type: ignore
import numpy as np # type: ignore
from numpy.random import seed # type: ignore
import cv2 # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from tqdm import tqdm # type: ignore
import pandas as pd # type: ignore
import math
import random
seed(42)

## **2 - Transforms**

In [ ]:
# Transforms for tf.data.Dataset inspired by: https://stackoverflow.com/questions/58270150/is-there-some-simple-way-to-apply-image-preprocess-to-tf-data-dataset
def transformTrainData(image, label):
    '''
    See the documentation for tf.image for more information on the transformations: 
    https://www.tensorflow.org/api_docs/python/tf/image
    '''
    # Performs scaling. NOTE: FEEL FREE TO MODIFY. SEE DOCUMENTATION ABOVE TO FIND MORE TRANSFORMATIONS.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.rot90(image, k = random.randint(0, 3))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

def transformValData(image, label):
    '''
    See the documentation for tf.image for more information on the transformations:
    https://www.tensorflow.org/api_docs/python/tf/image
    '''
    # Performs scaling
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label

## **3 - Hyperparamters** (_feel free to adjust_)

In [ ]:
# Specify a splitting for our train, val, and test data, and other hyperparameters
val_split = 0.2
# Image size
img_size = 64
# Batch size. This is the number of images that will be fed into the model at once. Adjust based on your memory capacity.
batch_size = 32
# Color mode
color = 'rgb' # 'grayscale'
# Seed for reproducibility
seed = 42
# Number of epochs
epochs = 10
# Loss function
loss = 'categorical_crossentropy'#'sparse_categorical_crossentropy' # 'categorical_crossentropy' (one-hot encoded labels) or 'sparse_categorical_crossentropy' (integer labels
# Optimizer
optimizer = 'adam' # 'sgd'
# Regularization parameters
wd = None
# momentum is a hyperparameter for the optimizer SGD that helps accelerate the convergence of the model
momentum = 9e-06
# learning rate is a hyperparameter that controls how much we are adjusting the weights of our network with respect the loss gradient
lr = 0.001
# Apply softmax to the output layer?
from_logits = False
# Metrics to track
metrics = ['accuracy'] # 'accuracy', 'f1_score', 'precision', 'recall'
# number of classes
num_classes = 29

## **4 - Preprocessing**

In [ ]:
# The data directories
data_path = "./data/asl_alphabet_train/asl_alphabet_train/"
test_path = "./data/asl_alphabet_test/asl_alphabet_test/"

In [ ]:
# Load the data into a tf.data.Dataset
train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(data_path, 
                                                                         labels = 'inferred', 
                                                                         label_mode = 'int' if loss == 'sparse_categorical_crossentropy' else 'categorical', 
                                                                         color_mode = color, 
                                                                         batch_size = batch_size, 
                                                                         image_size = (img_size, img_size), 
                                                                         shuffle = True, 
                                                                         seed = seed, 
                                                                         validation_split = val_split, 
                                                                         subset = "both")

In [ ]:
# Map the transformations to the dataset
train_dataset = train_dataset.map(transformTrainData)
val_dataset = val_dataset.map(transformValData)

## **5 - Model**

In [ ]:
# Load the pre-trained ResNet50 model
base_model = tf.keras.applications.ResNet50(
    include_top=False, #True
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_size, img_size, 3 if color == 'rgb' else 1),
    pooling=None,
    classifier_activation='softmax'
)
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)  
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)  
resnet = tf.keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
# Inspection of the model
resnet.summary()

# **5 - Other utilities**

In [ ]:
def optimizerFactory(optim: str, lr: float, momentum: float, wd: float):
    '''
    Choose an optimizer based on the passed in string.
    '''
    if optim == 'sgd':
        return tf.keras.optimizers.SGD(learning_rate = lr, momentum = momentum, decay = wd)
    elif optim == 'adam':
        return tf.keras.optimizers.Adam(learning_rate = lr)
    else:
        raise NotImplementedError(f'Optimizer {optim} not implemented')

In [ ]:
def lossFactory(loss: str, from_logits: bool = False):
    if loss == 'sparse_categorical_crossentropy':
        return tf.keras.losses.SparseCategoricalCrossentropy(from_logits = from_logits)
    elif loss == 'categorical_crossentropy':
        return tf.keras.losses.CategoricalCrossentropy(from_logits = from_logits)
    else:
        raise NotImplementedError(f'Loss {loss} not implemented')

## **6 - Training**

In [ ]:
metrics=['accuracy']#, tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score(average='macro'),]

In [ ]:
model = resnet

In [ ]:
# We specify our optimizer and loss function, and compile the model with the metrics we want to track
optimizer = optimizerFactory(optimizer, lr, momentum, wd)
loss = lossFactory(loss, from_logits)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = metrics) #optimizer, #loss

## **RUN THIS NEXT CELL AT YOUR OWN RISK. PLEASE USE A GPU.**

In [ ]:
model.optimizer.get_config()

In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs = epochs)

At the time of submission, we are still in the process of training. We will be running a script version of this jupyter notebook on a HPC overnight. 